In [2]:
# RNN(Long Short-Term Memory, LSTM)

In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import os
%matplotlib inline

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def mergeData():
    SaveFile_Name = 'data.csv'
    file_list = os.listdir('data')
    df = pd.read_csv('data'+'\\'+file_list[0])
    df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False)
    for i in range(1,len(file_list)):
        df = pd.read_csv('data'+'\\'+file_list[i])
        df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False, header=False, mode='a+')

In [5]:
def readData():
    train = pd.read_csv("data.csv")
    return train

In [6]:
def changeYear(data):
    for i in range(0,data.shape[0]):
        Date=data["日期"][i].split('/')
        year,month,date=Date[0],Date[1],Date[2]
        year=str(int(year)+1911)
        data.loc[i,"日期"]=year+'/'+month+'/'+date
    return data

In [7]:
# Augment Features
def augFeatures(data):
  data["日期"] = pd.to_datetime(data["日期"])
  data["年"] = data["日期"].dt.year
  data["月"] = data["日期"].dt.month
  data["日"] = data["日期"].dt.day
  data["第幾日"] = data["日期"].dt.dayofweek
  return data

In [8]:
def normalize(data):
    for i in range(0,data.shape[0]):
        if data["漲跌價差"][i]=='X0.00':
            data.loc[i,"漲跌價差"]=str(int(data["收盤價"][i])-int(data["收盤價"][i-1]))
    data=data.drop(["日期"], axis=1)
    data=data.drop(["成交股數"], axis=1)
    data=data.drop(["成交金額"], axis=1)
    data=data.drop(["漲跌價差"], axis=1)
    data=data.drop(["成交筆數"], axis=1)
    data=data.convert_objects(convert_numeric=True)
    datanormalize=data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return datanormalize

In [9]:
def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["開盤價"]))
  return np.array(X_train), np.array(Y_train)

In [10]:
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [11]:
# 將Training Data取一部份當作Validation Data
def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

In [12]:
def buildOneToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
  # output shape: (1, 1)
  model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [19]:

mergeData()
train=readData()
train=changeYear(train)
train=augFeatures(train)
train=normalize(train)
train_x, train_y = buildTrain(train, 1, 1)
train_x, train_y = shuffle(train_x, train_y )

# split training data and validation data
train_x, train_y , val_x, val_y = splitData(train_x, train_y , 0.1)
train_y = train_y[:,np.newaxis]
val_y = val_y[:,np.newaxis]

model = buildOneToOneModel(train_x.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")

model.fit(train_x, train_y, epochs=1000, batch_size=50, validation_data=(val_x, val_y), callbacks=[callback])


C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 8))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1, 10)             760       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 1, 1)              11        
Total params: 771
Trainable params: 771
Non-trainable params: 0
_________________________________________________________________
Train on 5571 samples, validate on 618 samples
Epoch 1/1000
5571/5571 [==============================] - 1s 144us/step - loss: 0.0196 - val_loss: 0.0029
Epoch 2/1000
5571/5571 [==============================] - 0s 32us/step - loss: 7.4295e-04 - val_loss: 2.2842e-04
Epoch 3/1000
5571/5571 [==============================] - 0s 27us/step - loss: 2.2631e-04 - val_loss: 1.3783e-04
Epoch 4/1000
5571/5571 [==============================] - 0s 26us/step - loss: 1.6550e-04 - val_loss: 9.5375e-05
Epoch 5/1000
5571/5571 [========================

5571/5571 [==============================] - 0s 27us/step - loss: 1.0628e-04 - val_loss: 6.0358e-05
Epoch 68/1000
5571/5571 [==============================] - 0s 25us/step - loss: 1.0273e-04 - val_loss: 5.8279e-05
Epoch 69/1000
5571/5571 [==============================] - 0s 25us/step - loss: 1.0285e-04 - val_loss: 5.9159e-05
Epoch 70/1000
5571/5571 [==============================] - 0s 27us/step - loss: 1.0260e-04 - val_loss: 5.3375e-05
Epoch 71/1000
5571/5571 [==============================] - 0s 26us/step - loss: 1.0387e-04 - val_loss: 5.5615e-05
Epoch 72/1000
5571/5571 [==============================] - 0s 25us/step - loss: 1.0201e-04 - val_loss: 5.5090e-05
Epoch 73/1000
5571/5571 [==============================] - 0s 25us/step - loss: 1.0228e-04 - val_loss: 5.3439e-05
Epoch 74/1000
5571/5571 [==============================] - 0s 28us/step - loss: 1.0156e-04 - val_loss: 6.0489e-05
Epoch 75/1000
5571/5571 [==============================] - 0s 30us/step - loss: 1.0021e-04 - val_loss:

In [20]:
scores= model.evaluate(val_x, val_y,verbose=1)
print(scores)

618/618 [==============================] - 0s 16us/step
4.897444198576019e-05
